In [4]:
from abc import ABC, abstractmethod

import os
from tqdm import tqdm
import math 
import time

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from scipy import stats
from scipy.special import digamma

import torch
from torch.distributions import Beta, kl_divergence
from torch.nn.functional import log_softmax
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, TensorDataset

from bayes_opt import BayesianOptimization

In [5]:
SEED = 1
np.random.seed(SEED)
torch.manual_seed(SEED)

In [6]:
### these params control the generation scheme (2/10 generation)
rho        = 0.8    # polarization
pop_size   = 50000  # num individuals
epsilon    = 0.05   # expected prop of speech consisting of neutral words
pi         = 0.5    # pi == 0.5 => beta mixture symmetrical (choose beta1 with prob pi = 0.5)
speech_len = 50     # words per speech
lex_size   = 3      # lexicon size 

In [7]:
def generate(rho=rho, N=pop_size, epsilon=epsilon, pi=pi, speech_len=speech_len, lex_size=lex_size, verbose='low'):
    """
    Uses 2/10 generation scheme to generate N samples.

    Returns:
        (X, y), (a, b, rho, epsilon, u, lex_size)
        X.size() == [N, S] is a vector of sequences 
        y.size() == [N] is a vector of political parties
        u.shape  == (N,) is a vector of individual stances
        rho is true polarization
        epsilon is expected prop of neutral words
        a, b are true alpha/beta for beta mixture model
    """
    start = time.time()
    if verbose in ['mid', 'high']:
        print(f'Beginning Data Generation...')
        print(f'=' * 20)
        
    ### get beta mixture model params 
    sigma = 0.175 * (rho ** 2) - 0.3625 * rho + 0.1875
    a     = rho * ((rho * (1 - rho)) / sigma - 1)
    b     = (1 - rho) * ((rho * (1 - rho)) / sigma - 1)

    if verbose in ['mid', 'high']:
        print(f'Lex Size: {lex_size}, Rho: {rho}, N: {pop_size}')
        print(f'epsilon: {epsilon}, pi: {pi}, S: {speech_len}\n')
        print(f"True Alpha: {a}")
        print(f"True Beta: {b}\n")

    mean = a / (a + b)
    var  = a * b / ((a + b)**2 * (a + b + 1))

    if abs(mean - rho) > 10e-15:
        print(f'Mean: {mean}')
        print(f"Rho: {rho}")
        raise AssertionError(f"Mean of BMM params should be rho")

    if abs(var - sigma) > 10e-15:
        print(f'Var: {var}')
        print(f'Sigma: {sigma}')
        raise AssertionError(f"Var of BMM params should be sigma")

    ### u ~ pi Beta(a, b) + (1 - pi) Beta(b, a)
    weights = [pi, 1-pi]
    mixture_samples = np.random.choice([0, 1], size=N, p=weights)

    u = 2 * np.where(mixture_samples == 1, stats.beta.rvs(a, b, size=N), stats.beta.rvs(b, a, size=N)) - 1

    if u.shape != (N,):
        raise AssertionError(f"u.shape should be (N,)")

    if verbose == 'high':
        print(f'mixture samples: {mixture_samples[:3]}')
        print(f'u samples: {u[:3]}')

    ### y = 1(u >= 0)
    y = (u >= 0).astype(int)

    if verbose == 'high':
        print(f'y samples: {y[:3]}\n')

    ### phi is a prob matrix that is a function of u, epsilon
    left_prob    = (1 - (u+1)/2) * (1 - epsilon) / lex_size
    right_prob   = (u+1)/2 * (1 - epsilon) / lex_size
    neutral_prob = np.repeat(epsilon, N) / lex_size
    phi          = np.array([left_prob] * lex_size + [right_prob] * lex_size + [neutral_prob] * lex_size).T
    
    if verbose == 'high':
        for i in range(3):
            print(f'P(L) = {phi[i,0]}, P(R): {phi[i, lex_size]}, P(N): {phi[i,-1]}')

    if phi.shape != (N, 3 * lex_size):
        raise AssertionError(f'phi.shape should be (N, V) == (N, 3L)')

    if abs(sum(phi[0]) - 1) > 10e-5:
        raise AssertionError(f'rows of phi should sum to 1')
    
    X = np.array([stats.multinomial.rvs(n=speech_len, p=phi[i, :]) for i in range(N)])

    if verbose == 'high':
        print(f'X samples (counts):\n {X[:3]}\n')
    
    if X.shape != (N, 3 * lex_size):
        raise AssertionError(f'X.shape should be (N, V) == (N, 3L)')

    if X[:5].sum() != speech_len * 5:
        raise AssertionError(f'rows of phi should sum to 1')

    expanded_rows = []
    for row in X:
        expanded_row = np.repeat(np.arange(len(row)), row)
        np.random.shuffle(expanded_row)
        expanded_rows.append(expanded_row)

    X = np.array(expanded_rows, dtype=np.int64)

    if X.shape != (N, speech_len):
        raise AssertionError(f'X.shape should be (N, S)')
    
    if verbose == 'high':
        print(f'X samples (sequences):\n {X[:3]}\n')

    X = torch.from_numpy(X).to(torch.float32)
    y = torch.from_numpy(y).to(torch.float32)

    known   = (X, y)
    unknown = (a, b, rho, epsilon, u, lex_size)

    if verbose in ['mid', 'high']:
        print('=' * 20)
        print(f'Generation Time: {round(time.time() - start, 3)} seconds for {N} samples.')

    return known, unknown

known, unknown = generate(verbose='high')
X, y = known
a, b, rho, epsilon, u, lex_size = unknown

Beginning Data Generation...
Lex Size: 3, Rho: 0.8, N: 50000
epsilon: 0.05, pi: 0.5, S: 50

True Alpha: 12.673684210526261
True Beta: 3.1684210526315644

mixture samples: [0 1 0]
u samples: [-0.71319006  0.57688516 -0.46523368]
y samples: [0 1 0]

P(L) = 0.2712550923688634, P(R): 0.04541157429780326, P(N): 0.016666666666666666
P(L) = 0.06699318292320035, P(R): 0.24967348374346632, P(N): 0.016666666666666666
P(L) = 0.23199533330535926, P(R): 0.08467133336130739, P(N): 0.016666666666666666
X samples (counts):
 [[16 13 14  2  1  2  0  0  2]
 [ 2  3  3 12 13 15  1  0  1]
 [15 15  8  3  4  5  0  0  0]]

X samples (sequences):
 [[0 2 2 2 2 0 0 2 0 1 0 4 0 1 1 3 2 1 2 0 0 5 0 1 0 0 1 2 1 1 0 8 2 2 0 1
  1 3 1 1 2 0 8 2 1 2 2 5 0 0]
 [3 4 5 4 5 5 3 3 1 6 5 0 3 8 4 4 3 5 0 3 5 5 3 4 5 4 1 3 5 5 5 3 2 2 2 5
  4 5 4 3 4 1 4 3 5 4 3 4 5 4]
 [1 5 1 0 1 1 1 2 1 0 5 4 4 0 0 1 0 0 3 0 1 0 0 0 5 1 0 5 3 5 0 2 1 1 2 1
  0 2 3 0 1 4 2 0 2 2 1 1 4 2]]

Generation Time: 2.07 seconds for 50000 samples.


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score


In [7]:
X_np = X.numpy()
y_np = y.numpy()

# Initialize the classifier
gb_clf = GradientBoostingClassifier()

# Setup cross-validation
kf = StratifiedKFold(n_splits=5)

# Get predicted probabilities for each sample across all folds using predict_proba
predicted_probs = cross_val_predict(gb_clf, X_np, y_np, cv=kf, method='predict_proba')

# Calculate the mean predicted probabilities (p_hats)
p_hats = predicted_probs[:, 1]  # Get probabilities for class 1

# Compute mean accuracy
predicted_class = predicted_probs[:, 1] > 0.5  # Convert probabilities to class predictions (threshold 0.5)
mean_accuracy = accuracy_score(y_np, predicted_class)

# Print results
print(f"Mean Accuracy: {mean_accuracy}")
print(f"p_hats (mean predicted probabilities for class 1): {p_hats}")

Mean Accuracy: 0.98782
p_hats (mean predicted probabilities for class 1): [6.22585932e-04 9.96754482e-01 4.88675910e-03 ... 9.97225659e-01
 1.01027479e-01 9.97233834e-01]


In [5]:
grid_size  = 200 ### discretize integral into grid_size linearly spaced pts
batch_size = 100 ### num samples processed simultaneously
num_epochs = 200 ### number epochs
lr         = 0.1 ### learning rate

def compute_nll(x_batch, y_batch, log_alpha, log_beta, W):
    """
    Parameters:
        (x_batch, y_batch) a batch of B (x,y) samples 
        (log_alpha, log_beta, W) trainable parameters

    Returns:
        A scaler quantity s.t. calling .backward() will compute
        grad_{theta} (L(theta))
    """
    raise NotImplementedError

def train(X, y, log_alpha=None, log_beta=None, W=None, num_epochs=num_epochs, batch_size=batch_size, grid_size=grid_size, lr=lr, verbose=True, threshold=0.00001, patience=5, speech_len=speech_len, lex_size=lex_size):
    if verbose:
        print(f'Beginning Inference...')
        print(f'=' * 20)
        print(f'Hyperparameters:')
        print(f'Dataset Length: {len(X)}')
        print(f'Number Epochs: {num_epochs}')
        print(f'Batch Size: {batch_size}')
        print(f'Grid Size: {grid_size}')
        print(f"Learning Rate: {lr}")
        print(f"Improvement Threshold: {threshold}")
        print(f"Patience: {patience}")
        print(f'Speech Len: {speech_len}')
        print('=' * 20)

    ### not strictly necessary 
    assert len(X) % batch_size == 0
    
    start = time.time()

    dataset    = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    X_batch, y_batch = list(dataloader)[0]

    if list(X_batch.size()) != [batch_size, speech_len]:
        raise AssertionError(f"X_ex.size() should be [batch_size, S], got {X_batch.size()}")

    if list(y_batch.size()) != [batch_size]:
        raise AssertionError(f"y_ex.size() should be [batch_size], got {y_batch.size()}")

    if log_alpha is None:
        log_alpha = torch.normal(0, math.sqrt(3), size=(1,), requires_grad=True)

    if log_beta is None:
        log_beta  = torch.normal(0, math.sqrt(3), size=(1,), requires_grad=True)

    if W is None:
        W = torch.normal(0, 1, size=(3 * lex_size,), requires_grad=True)

    print(f'Initial Parameters:')
    print(f'Alpha: {torch.exp(log_alpha).item()}')
    print(f"Beta: {torch.exp(log_beta).item()}")
    print(f"W: {W.data.tolist()}")

    optimizer = SGD([log_alpha, log_beta, W], lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    # Early stopping logic
    best_nll = float('inf')
    epochs_without_improvement = 0

    for epoch in tqdm(range(num_epochs)):
        if verbose:
            print(f'Epoch {epoch+1} of {num_epochs}')
        ### only step after each epoch
        optimizer.zero_grad() 
        nll = torch.tensor(0.)
        
        for x_batch, y_batch in dataloader:
            nll += compute_nll(x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size, speech_len, lex_size)
        
        nll = nll / len(dataloader.dataset)
        nll.backward()
        print('=' * 20)
        print(f'Log Alpha Grad: {log_alpha.grad}')
        print(f'Log Beta Grad: {log_beta.grad}')
        print(f'W Grad: {W.grad}')
        print('=' * 20)
        optimizer.step()
        scheduler.step()

        current_nll = nll.item()
        if current_nll < best_nll - threshold:
            best_nll = current_nll
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if verbose:
            print(f'Epoch {epoch+1}, NLL: {round(nll.item(), 3)}, Alpha: {round(np.exp(log_alpha.item()),3)}, Beta: {round(np.exp(log_beta.item()),3)}, W: {W.tolist()}')
            print('=' * 20)

        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    final_alpha = np.exp(log_alpha.item())
    final_beta  = np.exp(log_beta.item())

    if verbose:
        print(f"Training Took {round(time.time() - start, 2)} Seconds.")
        
    print(f"Trained Params: alpha: {final_alpha}, beta: {final_beta}, W: {W.tolist()}")

    return best_nll, final_alpha, final_beta, W

In [6]:
### numerically stable factorial
factorial = lambda x : torch.exp(torch.lgamma(x+1))

assert factorial(torch.tensor(3.)) == torch.tensor(6.)
print(f"3.0001! = {factorial(torch.tensor(3.001)).item()}")

### notice this is infinity, so we can't call factorial on negative stuff
print(f"-1! = {factorial(torch.tensor(-1.)).item()}")

def compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size, speech_len, lex_size):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
        u_mat   (size [batch_size, grid_size])
        (log_alpha, log_beta, W) trainable params 
    
    Returns:
        torch.tensor (size [batch_size, grid_size]), where the (i,j)th element 
        is the log joint probability (log p(u, x, y; theta)). Here u is the (i,j)th
        element of u_mat, x is the ith row of x_batch, y is the ith element of y_batch

    """
    assert list(u_mat.size())   == [batch_size, grid_size] 
    assert list(x_batch.size()) == [batch_size, speech_len]
    assert list(y_batch.size()) == [batch_size]

    if not torch.all((u_mat >= 0).float() == y_batch.unsqueeze(1)).item():
        raise ValueError(f"u_mat is incompatible with y_batch.")

    ### log prior: log p(u) = log(1/4 Beta((u+1)/2; a, b) + 1/4 Beta((u+1)/2; b, a))
    ###                     = log(1/4) + log(Beta((u+1)/2; a, b) + Beta((u+1)/2; b, a))
    alpha = torch.exp(log_alpha)
    beta  = torch.exp(log_beta)
        
    beta_dist_ab = Beta(alpha, beta)
    beta_dist_ba = Beta(beta, alpha)

    ### see 3-6 testing.ipynb test1 to see how log_prob broadcasts
    log_beta_prob_ab = beta_dist_ab.log_prob((u_mat + 1) / 2)
    log_beta_prob_ba = beta_dist_ba.log_prob((u_mat + 1) / 2)

    prior = torch.log(torch.exp(log_beta_prob_ab) + torch.exp(log_beta_prob_ba)) + torch.log(torch.tensor(0.25))

    assert list(prior.size()) == [batch_size, grid_size]
  
    ### log likelihood: log p(x, y | u) = dot(x, log Softmax(Wu)) + log(S!/(x1! x2! x3!)) 

    ### step 1: compute Wu (see 3-6 testing.ipynb test2 to see this logic)
    W_expanded = W.unsqueeze(0).unsqueeze(0)
    u_expanded = u_mat.unsqueeze(2)
    Wu = torch.matmul(u_expanded, W_expanded) 

    assert list(W_expanded.size()) == [1,1,3 * lex_size]
    assert list(u_expanded.size()) == [batch_size, grid_size, 1]
    assert list(Wu.size()) == [batch_size, grid_size, 3 * lex_size]

    ### step 2: dot(x, log Softmax(Wu)) (see 3-6 testing.ipynb test3 to see this logic)
    #print(x_batch[:2]) # B x S 
    ones = torch.ones_like(x_batch, dtype=torch.float)
    #print(ones[:2]) # B x S 
    # Assuming x_batch needs to be adjusted by -1 for 0-indexing
    x_batch_trans = torch.zeros(batch_size, 3 * lex_size, dtype=torch.float)

    ones = torch.ones_like(x_batch, dtype=torch.float)
    x_batch_trans.scatter_add_(dim=1, index=x_batch.long(), src=ones)
    likelihood = (x_batch_trans.unsqueeze(1) * log_softmax(Wu, dim=2)).sum(dim=2)

    ### ensure not taking negative factorials
    assert torch.all(x_batch >= 0)

    assert list(likelihood.size()) == [batch_size, grid_size]
    return prior + likelihood


def compute_nll(x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size, speech_len, lex_size):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
        log_alpha, log_beta, W (trainable params)
        grid_size (int) number of lin spaced points to discretize integral into
    
    Returns:
        quantity s.t. .backward() would be the gradient of the nll
    """
    ### see 3-6 testing.ipynb test5 to see this logic
    u_mat = torch.empty(batch_size, grid_size)
    u_mat[y_batch == 1] = torch.linspace(1/(grid_size+1), 1-1/(grid_size+1), grid_size).repeat((y_batch == 1).sum(), 1)
    u_mat[y_batch == 0] = torch.linspace(-1+1/(grid_size+1), -1/(grid_size+1), grid_size).repeat((y_batch == 0).sum(), 1)

    assert list(u_mat.size()) == [batch_size, grid_size]

    log_joint = compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size, speech_len, lex_size)
    assert list(log_joint.size()) == [batch_size, grid_size]

    ### see 3-6 testing.ipynb test6 to see this logic
    max_log_prob = torch.max(log_joint, dim=1, keepdim=True)[0]
    joint_probs = torch.exp(log_joint - max_log_prob)
    posterior = joint_probs / joint_probs.sum(dim=1, keepdim=True) 
    # detach posterior (see 3-6 testing.ipynb test7 to verify this works)
    weighted_log_joint = posterior.detach() * log_joint
    nll = -weighted_log_joint.sum()

    return nll

3.0001! = 6.007541656494141
-1! = inf


In [7]:
rho = 0.8
N = 50000
epsilon = 0.
lex_size = 5
speech_len = 50

known, unknown = generate(rho=rho, N=N, verbose='high', epsilon=epsilon, lex_size=lex_size, speech_len=speech_len)
X, y = known 
a, b, rho, epsilon, u, lex_size = unknown

Beginning Data Generation...
Lex Size: 5, Rho: 0.8, N: 50000
epsilon: 0.0, pi: 0.5, S: 50

True Alpha: 12.673684210526261
True Beta: 3.1684210526315644

mixture samples: [0 0 0]
u samples: [-0.28357936 -0.52389393 -0.25908543]
y samples: [0 0 0]

P(L) = 0.12835793622206623, P(R): 0.07164206377793378, P(N): 0.0
P(L) = 0.15238939281614144, P(R): 0.047610607183858546, P(N): 0.0
P(L) = 0.12590854271088564, P(R): 0.07409145728911436, P(N): 0.0
X samples (counts):
 [[ 6  9  7  6  3  4  5  6  2  2  0  0  0  0  0]
 [ 5 10  7  8 11  2  2  1  1  3  0  0  0  0  0]
 [ 5  7  7  4  5  6  3  5  3  5  0  0  0  0  0]]

X samples (sequences):
 [[2 3 1 0 1 1 5 6 5 3 7 3 2 2 3 2 3 7 7 6 4 1 1 0 5 0 5 6 4 6 2 9 0 6 1 8
  8 7 2 0 3 2 1 4 1 7 9 1 7 0]
 [6 2 1 1 9 1 4 4 0 1 4 3 1 0 5 4 2 1 6 1 2 4 4 2 4 4 7 0 4 4 2 3 8 2 3 3
  1 1 5 4 2 0 3 3 9 0 9 3 3 1]
 [8 5 5 2 9 0 3 0 5 7 9 4 7 7 5 8 2 7 4 3 6 2 1 2 3 6 9 3 4 4 5 2 4 0 1 8
  7 9 0 1 1 5 1 2 2 6 1 9 0 1]]

Generation Time: 2.066 seconds for 50000 samples.

Testing if magnitude of starting params matter when initial params are bigger

In [11]:
best_nll, final_alpha, final_beta, W = train(X, y, patience=100, grid_size=200, num_epochs=200,
                                            lr=.1, speech_len=speech_len, lex_size=lex_size, 
                                            log_alpha= torch.tensor(np.log(12.67 + 0.7), requires_grad=True),
                                            log_beta= torch.tensor(np.log(3.168 + 0.7), requires_grad=True))
                                    

Beginning Inference...
Hyperparameters:
Dataset Length: 50000
Number Epochs: 200
Batch Size: 100
Grid Size: 200
Learning Rate: 0.1
Improvement Threshold: 1e-05
Patience: 100
Speech Len: 50
Initial Parameters:
Alpha: 13.369999999999996
Beta: 3.8680000000000003
W: [-0.8029409646987915, 0.2365746796131134, 0.2856927514076233, 0.6898148655891418, -0.6330540180206299, 0.8794752955436707, -0.6841781735420227, 0.45329079031944275, 0.29115796089172363, -0.8317165970802307, -0.5525082945823669, 0.6354773044586182, -0.39681580662727356, -0.6570598483085632, -1.6427524089813232]


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1 of 200


  0%|          | 1/200 [00:11<38:39, 11.65s/it]

Log Alpha Grad: 2.3922102321122476
Log Beta Grad: -1.7699673850672748
W Grad: tensor([ 0.4093,  0.8110,  0.8409,  1.0852,  0.4642, -0.7275, -1.4446, -0.9908,
        -1.0771, -1.4860,  0.3603,  1.0042,  0.4246,  0.3196,  0.0069])
Epoch 1, NLL: 136.245, Alpha: 10.525, Beta: 4.617, W: [-0.8438687920570374, 0.1554744839668274, 0.2016005665063858, 0.5812910199165344, -0.6794707179069519, 0.9522245526313782, -0.5397146940231323, 0.5523750185966492, 0.39886826276779175, -0.6831145882606506, -0.5885363221168518, 0.5350614786148071, -0.43927285075187683, -0.6890159249305725, -1.6434441804885864]
Epoch 2 of 200


...

Log Alpha Grad: 0.05258326463008554
Log Beta Grad: 0.008691319620795036
W Grad: tensor([ 3.7864e-02,  3.7936e-02,  3.8017e-02,  3.7926e-02,  3.7985e-02,
        -3.7698e-02, -3.7683e-02, -3.7637e-02, -3.7657e-02, -3.7676e-02,
        -2.7359e-04, -2.0283e-05, -2.3568e-04, -2.9957e-04, -5.4943e-04])
Epoch 77, NLL: 118.978, Alpha: 6.457, Beta: 1.459, W: [-1.970669150352478, -1.975184440612793, -1.9794576168060303, -1.976011037826538, -1.9766480922698975, 1.617817759513855, 1.611210584640503, 1.6110402345657349, 1.6096266508102417, 1.6133973598480225, -0.18293100595474243, -0.1826639175415039, -0.18289101123809814, -0.18295839428901672, -0.18322183191776276]
Epoch 78 of 200


 39%|███▉      | 78/200 [13:12<20:31, 10.09s/it]

Log Alpha Grad: 0.05133979233742328
Log Beta Grad: 0.00826455034656985
W Grad: tensor([ 3.7247e-02,  3.7318e-02,  3.7398e-02,  3.7308e-02,  3.7367e-02,
        -3.7106e-02, -3.7091e-02, -3.7046e-02, -3.7066e-02, -3.7084e-02,
        -2.4719e-04, -1.6721e-05, -2.1267e-04, -2.7084e-04, -4.9814e-04])
Epoch 78, NLL: 118.977, Alpha: 6.424, Beta: 1.458, W: [-1.9743938446044922, -1.9789161682128906, -1.9831973314285278, -1.9797418117523193, -1.9803848266601562, 1.6215283870697021, 1.614919662475586, 1.614744782447815, 1.6133332252502441, 1.6171057224273682, -0.18290628492832184, -0.1826622486114502, -0.18286974728107452, -0.1829313039779663, -0.18317201733589172]
Epoch 79 of 200


 39%|███▉      | 78/200 [13:12<20:39, 10.16s/it]


KeyboardInterrupt: 

In [12]:
best_nll, final_alpha, final_beta, W = train(X, y, patience=100, grid_size=200, num_epochs=200,
                                            lr=.1, speech_len=speech_len, lex_size=lex_size, 
                                            log_alpha= torch.tensor(np.log(12.67 + -.5), requires_grad=True),
                                            log_beta= torch.tensor(np.log(3.168 + 0.1), requires_grad=True),
                                            W = torch.normal(10, 2, size=(3 * lex_size,), requires_grad=True))

Beginning Inference...
Hyperparameters:
Dataset Length: 50000
Number Epochs: 200
Batch Size: 100
Grid Size: 200
Learning Rate: 0.1
Improvement Threshold: 1e-05
Patience: 100
Speech Len: 50
Initial Parameters:
Alpha: 12.170000000000002
Beta: 3.268
W: [11.960583686828613, 11.142026901245117, 12.866023063659668, 13.337730407714844, 13.613570213317871, 8.694578170776367, 12.097506523132324, 10.995043754577637, 10.773017883300781, 6.944344997406006, 10.778311729431152, 8.514144897460938, 9.893498420715332, 10.913046836853027, 11.223490715026855]


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1 of 200


  0%|          | 1/200 [00:14<47:20, 14.27s/it]

Log Alpha Grad: 5.239091613097167
Log Beta Grad: -3.165634268340038
W Grad: tensor([ 0.0724,  0.0668,  0.0805,  0.0846,  0.0877, -0.1184, -0.1077, -0.1123,
        -0.1127, -0.1199,  0.0398,  0.0221,  0.0323,  0.0410,  0.0439])
Epoch 1, NLL: 138.867, Alpha: 7.207, Beta: 4.485, W: [11.953339576721191, 11.135344505310059, 12.857972145080566, 13.329269409179688, 13.604802131652832, 8.706415176391602, 12.10827922821045, 11.006277084350586, 10.784293174743652, 6.956336975097656, 10.774333000183105, 8.511940002441406, 9.890268325805664, 10.908946990966797, 11.219100952148438]
Epoch 2 of 200


...

Log Alpha Grad: -0.2448358749569914
Log Beta Grad: -0.24483587567879903
W Grad: tensor([ 0.0748,  0.0696,  0.0822,  0.0859,  0.0885, -0.1225, -0.1120, -0.1164,
        -0.1168, -0.1241,  0.0420,  0.0248,  0.0348,  0.0432,  0.0459])
Epoch 20, NLL: 135.803, Alpha: 8.978, Beta: 8.978, W: [11.817793846130371, 11.00838565826416, 12.710184097290039, 13.17557430267334, 13.446688652038574, 8.921286582946777, 12.306028366088867, 11.211018562316895, 10.989681243896484, 7.174106121063232, 10.704062461853027, 8.470945358276367, 9.832207679748535, 10.836716651916504, 11.142239570617676]
Epoch 21 of 200


 10%|█         | 21/200 [03:42<29:34,  9.91s/it]

Log Alpha Grad: -0.24415359290712588
Log Beta Grad: -0.24415359301020703
W Grad: tensor([ 0.0753,  0.0700,  0.0828,  0.0865,  0.0892, -0.1237, -0.1131, -0.1175,
        -0.1179, -0.1254,  0.0427,  0.0252,  0.0354,  0.0439,  0.0466])
Epoch 21, NLL: 135.787, Alpha: 9.2, Beta: 9.2, W: [11.810264587402344, 11.001385688781738, 12.701905250549316, 13.166923522949219, 13.437766075134277, 8.933659553527832, 12.317336082458496, 11.222765922546387, 11.00147533416748, 7.186645030975342, 10.69979476928711, 8.46842098236084, 9.828669548034668, 10.832330703735352, 11.13757610321045]
Epoch 22 of 200


 10%|█         | 21/200 [03:47<32:16, 10.82s/it]


KeyboardInterrupt: 

In [13]:
best_nll, final_alpha, final_beta, W = train(X, y, patience=100, grid_size=200, num_epochs=200,
                                            lr=.1, speech_len=speech_len, lex_size=lex_size, 
                                            log_alpha= torch.tensor(np.log(12.67/2), requires_grad=True),
                                            log_beta= torch.tensor(np.log(3.168/2), requires_grad=True),
                                            W = torch.normal(5, 2, size=(3 * lex_size,), requires_grad=True))

Beginning Inference...
Hyperparameters:
Dataset Length: 50000
Number Epochs: 200
Batch Size: 100
Grid Size: 200
Learning Rate: 0.1
Improvement Threshold: 1e-05
Patience: 100
Speech Len: 50
Initial Parameters:
Alpha: 6.335
Beta: 1.584
W: [7.939346790313721, 4.2098917961120605, 3.979785442352295, 7.232603549957275, 3.8148579597473145, 6.81782865524292, 2.8415327072143555, 3.649792194366455, 7.81660795211792, 1.30876886844635, 3.8690185546875, 3.18125319480896, 4.021854400634766, 3.7650866508483887, 5.767164707183838]


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1 of 200


  0%|          | 1/200 [00:10<35:21, 10.66s/it]

Log Alpha Grad: 2.7409296677660584
Log Beta Grad: -1.8574226545494417
W Grad: tensor([ 0.1757,  0.1105,  0.1088,  0.1601,  0.1070, -0.1697, -0.2052, -0.2003,
        -0.1523, -0.2136,  0.0506,  0.0394,  0.0532,  0.0489,  0.0867])
Epoch 1, NLL: 137.632, Alpha: 4.816, Beta: 1.907, W: [7.921778678894043, 4.198837757110596, 3.9689013957977295, 7.21658992767334, 3.8041574954986572, 6.834796905517578, 2.8620548248291016, 3.6698269844055176, 7.831838130950928, 1.3301278352737427, 3.863954782485962, 3.17730975151062, 4.0165300369262695, 3.760197639465332, 5.758491039276123]
Epoch 2 of 200


...

Log Alpha Grad: -0.2163934261185132
Log Beta Grad: -0.2163953408709197
W Grad: tensor([ 0.3175,  0.1540,  0.1488,  0.2770,  0.1445, -0.2451, -0.3902, -0.3706,
        -0.1774, -0.4182,  0.1005,  0.0770,  0.1061,  0.0968,  0.1795])
Epoch 30, NLL: 134.719, Alpha: 5.826, Beta: 5.826, W: [7.256275653839111, 3.824897289276123, 3.6035397052764893, 6.621913909912109, 3.4467573165893555, 7.426677227020264, 3.662523031234741, 4.441867828369141, 8.324426651000977, 2.173799753189087, 3.6568102836608887, 3.015826940536499, 3.798694372177124, 3.560204029083252, 5.401180744171143]
Epoch 31 of 200


 16%|█▌        | 31/200 [04:33<21:41,  7.70s/it]

Log Alpha Grad: -0.2130352680253515
Log Beta Grad: -0.21303623358622745
W Grad: tensor([ 0.3263,  0.1561,  0.1507,  0.2842,  0.1463, -0.2473, -0.4021, -0.3812,
        -0.1754, -0.4319,  0.1030,  0.0787,  0.1087,  0.0991,  0.1848])
Epoch 31, NLL: 134.637, Alpha: 5.952, Beta: 5.952, W: [7.223644256591797, 3.8092868328094482, 3.5884668827056885, 6.593498706817627, 3.4321300983428955, 7.451409339904785, 3.7027294635772705, 4.47998571395874, 8.341967582702637, 2.2169859409332275, 3.646512985229492, 3.007960557937622, 3.7878236770629883, 3.5502896308898926, 5.382702350616455]
Epoch 32 of 200


 16%|█▌        | 31/200 [04:41<25:34,  9.08s/it]


KeyboardInterrupt: 